# Setup

In [12]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [13]:
import google.generativeai as genai

genai.configure()

# Fundamentos de prompting

In [50]:
# Create the model
generation_config = {
  "temperature": 0,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}
model = genai.GenerativeModel('gemini-2.0-flash')

print(model.generate_content("The sky is").text)

The sky is **a vast expanse of atmosphere above the Earth, often appearing blue during the day.**

Of course, depending on the time of day, weather conditions, and location, the sky can be many other things:

*   **Blue:** During the day, due to Rayleigh scattering of sunlight.
*   **Gray:** When it's cloudy.
*   **Orange, Pink, or Red:** At sunrise and sunset.
*   **Black:** At night.
*   **Full of stars:** At night, away from light pollution.

What did you have in mind when you asked?



## Seamos un poco mas especificos

In [17]:
print(model.generate_content("""
Complete the sentence: 
The sky is """).text)

The sky is **blue**.



## Formateo de Prompts

In [18]:
print(model.generate_content("""
Q: What is prompt engineering?
A: """).text)

Prompt engineering is the art and science of crafting effective prompts for large language models (LLMs) to elicit desired responses. It involves carefully designing and refining the input text given to an LLM to guide its output and achieve a specific goal. Think of it as speaking the language the AI understands to get the best results.

Here's a breakdown of what that means:

*   **Crafting:** Prompt engineering isn't just about typing a question. It's about structuring the input strategically. This can involve using specific wording, formatting, keywords, and providing context.
*   **Effective Prompts:** The goal is to create prompts that are clear, concise, and unambiguous so the LLM understands what is being asked and how to respond.
*   **Large Language Models (LLMs):** LLMs are AI models trained on vast amounts of text data. They can generate text, translate languages, answer questions, and more. Examples include GPT-3, LaMDA, and others.
*   **Desired Responses:**  The whole po

Un prompt normalmente tiene los siguientes formatos:
```sh
<Question>?
```
Or
```sh
<Instruction>
```

Cabe aclarar que no es requerido usar este formato ni ningun otro. Siempre podemos adaptar el prompt de acuerdo a nuestras necesidades.

In [19]:
print(model.generate_content("""
This is awesome! // Positive
This is bad! // Negative
Wow that movie was rad! // Positive
What a horrible show! //""").text)

Negative



# Elementos de un prompt

* Instruccion: La tarea que queremos que haga el modelo.
* Contexto: Informacion adicional que puede ayudar al modelo a generar una mejor respuesta.
* Input: la pregunta o texto de entrada que queremos que el modelo responda.
* Indicador de salida: el tipo, o formato que queremos que el modelo use en su respuesta.

In [20]:
print(model.generate_content("""
Classify the text into neutral, negative, or positive.
Text: I think the food was okay.
Sentiment:
""").text)

Sentiment: Neutral



# Consejos Basicos

## Empieza sencillo:
* El prompting es un proceso iterativo y requiere mucha prueba y error para llegar a los resultados esperados. Siempre empieza en un entorno de pruebas sencillo.
* Empieza siempre con un prompt simple, y ve agregando cosas para que tu prompt vaya produciendo mejores resultados.
* Si tienes una tarea grande y compleja, trata de dividirla en tareas mas pequennas y sencillas.

## La instruccion
* Usa palabras simples que instruyan al modelo sobre lo que queremos lograr: *Write*, *Classify*, *Summarize*, *Translate*, *Order*.
* De nuevo el prompting es un proceso iterativo y necesitas experimentar para ver que combinacion de intrucciones, palabras clave y contextos te da los mejores resultados para tu tarea.
* Otra recomendacion es usar separadores y tags muy bien diferenciados.

  Por ejemplo:

In [46]:
model.generate_content("""
### Instruction ###
Translate the text below to spanish
### Task ###
text: "hello!"
""").text

'¡Hola!\n'

## Ser especifico

* Se debe ser muy especifico acerca de la tarea que se quiere realizar. Entre mas descriptivo y detallado sea el prompt, mejores seran los resultados.
* Ten en cuenta que aunque hoy dia los modelos tienen tamannos de contextos muy grandes, annadir demasiada informacion al prompt no siempre ayuda. Debes mantener un buen equilibrio entre la cantidad de informacion que le mandas al modelo, y la calidad de esta.

In [23]:
print(model.generate_content("""
Extract the name of places in the following text. 
Desired format:
Place: <comma_separated_list_of_places>
Input: "Although these developments are encouraging to researchers, much is still a mystery.
“We often have a black box between the brain and the effect we see in the periphery,”
says Henrique Veiga-Fernandes, a neuroimmunologist at the Champalimaud Centre for the Unknown
in Lisbon. “If we want to use it in the therapeutic context, we actually need to understand
the mechanism.“"
""").text)

Place: Lisbon



## Evitar imprecisiones

* Recuerda, es muy importante ser detallado y especifico, existe el riesgo de querer pasarse de listo, y esto puede llevar a prompts imprecisos.

In [24]:
print(model.generate_content("""
Explain the concept prompt engineering. Keep the explanation short, only a few sentences, and don't be too descriptive.
""").text)

Prompt engineering is the art and science of crafting effective instructions for AI models to elicit desired responses. It involves carefully designing and refining prompts to guide the model towards generating specific, accurate, and useful outputs. Think of it as strategically communicating with AI to get the best results.



Haz mejor esto:

In [25]:
print(model.generate_content("""
Use 2-3 sentences to explain the concept of prompt engineering to a high school student.
""").text)

Prompt engineering is like crafting the perfect question to get the best answer from a really smart computer program, like a chatbot. By carefully choosing the words and structure of your request, you can guide the AI to give you more accurate, creative, or useful results.



## Hacer o no hacer?

Un error común que todos cometemos cuando hacemos prompts, es decir que no queremos. En lugar de esto, debemos enfocarnos en lo que queremos lograr.

In [26]:
print(model.generate_content("""
The following is an agent that recommends movies to a customer.
Recommends three or four movies. DO NOT ASK FOR INTERESTS.
DO NOT ASK FOR PERSONAL INFORMATION.
Customer: Please recommend a movie based on my interests.
Agent:
""").text)

Okay! Based on what's popular and generally well-received right now, I'd recommend:

1.  **"Oppenheimer"**: A historical drama about the development of the atomic bomb.
2.  **"Barbie"**: A comedy fantasy film about Barbie and Ken navigating the real world.
3.  **"Spider-Man: Into the Spider-Verse"**: An animated superhero film with a unique visual style.
4.  **"Everything Everywhere All at Once"**: A mind-bending action comedy with a lot of heart.



Here is a better prompt:

In [27]:
print(model.generate_content("""
The following is an agent that recommends movies to a customer.
The agent is responsible to recommend a movie from the top global trending movies.
It should refrain from asking users for their preferences and avoid asking for
personal information. If the agent doesn't have a movie to recommend,
it should respond "Sorry, couldn't find a movie to recommend today.".

Customer: Please recommend a movie based on my interests.
Agent:
""").text)

Sorry, I am not allowed to ask for your interests. I would recommend "Oppenheimer" which is currently trending globally.



# Tecnicas de Prompting

## Zero-Shot Prompting
El prompt que usamos no contiene ejemplos o demostraciones, es muy efectivo en tareas sencillas, y normalmente siempre es nuestra primera opcion.

In [28]:
print(model.generate_content("""
Classify the text into neutral, negative or positive. Answer with a word.
Text: I think the vacation is okay.
Sentiment: """).text)

Neutral



## Few-Shot Prompting
Few-shot learning es una de las tecnicas basicas para proveer al modelo con in-context learning.
Es proveer ejemplos o demostraciones para que el modelo entienda mejor la tarea a la hora de generar una respuesta.

In [29]:
print(model.generate_content("""
A "whatpu" is a small, furry animal native to Tanzania. An example of a sentence that uses the word whatpu is:
We were traveling in Africa and we saw these very cute whatpus.
To do a "farduddle" means to jump up and down really fast. An example of a sentence that uses the word farduddle is:
""").text)

The little girl was so excited about the ice cream that she started to farduddle.



Otro ejemplo:

In [31]:
print(model.generate_content("""
This is awesome! // Negative
This is bad! // Positive
Wow that movie was rad! // Positive
What a horrible show! // """).text)

Negative



In [32]:
print(model.generate_content("""Positive
This is awesome! 
This is bad! Negative
Wow that movie was rad!
Positive
What a horrible show! -- """).text)

Negative



### Limitations

In [33]:
print(model.generate_content("""
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A: """).text)

Let's identify the odd numbers in the list: 15, 5, 13, 7, 1

Now, let's add them together: 15 + 5 + 13 + 7 + 1 = 41

So the answer is 41

A: 41



In [34]:
print(model.generate_content("""
Instruction:
The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The answer is False.
The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: The answer is True.
The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: The answer is True.
The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: The answer is False.
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A: """).text)

The answer is True.



## Chain of Thought Prompting

La cadena de pensamiento permite al modelo razonar a traves de pasos intermedios antes de proveer una respuesta.
Se puede combinar con few-shot prompting para obtener mejores resultados en tareas mas complejas.

In [35]:
print(model.generate_content("""
The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: Adding all the odd numbers (17, 19) gives 36. The answer is True.

The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: Adding all the odd numbers (11, 13) gives 24. The answer is True.

The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: Adding all the odd numbers (17, 9, 13) gives 39. The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A: """).text)

Adding all the odd numbers (15, 5, 13, 7, 1) gives 41. The answer is False.



Y funciona igual de bien con menos ejemplos

In [36]:
print(model.generate_content("""
The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A: """).text)

Adding all the odd numbers (15, 5, 13, 7, 1) gives 41. The answer is False.



El prompt: **Let's think step by step** => **Piensa paso a paso**

In [37]:
print(model.generate_content("""How many letters *r* contains the word *strawberry*?""").text)

The word "strawberry" contains two "r"s.



In [38]:
print(model.generate_content("""
How many letters *r* contains the word *strawberry*? Let's think step by step.""").text)

The word "strawberry" has the following letters: s, t, r, a, w, b, e, r, r, y.
The letter "r" appears three times in the word "strawberry".

So the answer is 3.



**Nota**: Hoy dia casi todos los modelos hacen CoT por defecto.

## Meta Prompting

Meta Prompting es una técnica avanzada que se centra en la estructura y sintaxis de las tareas en lugar de sus detalles específicos. Su objetivo es crear una forma más abstracta y estructurada de interactuar con modelos de lenguaje, priorizando patrones de información sobre el contenido tradicional.

### Caracteristicas
* Orientado a la estructura: Prioriza el formato y patrón de problemas por encima del contenido específico.
* Enfocado en la sintaxis: Usa la sintaxis como una plantilla para la respuesta esperada.
* Ejemplos abstractos: Se pueden usar ejemplos para ilustrar la estructura del problema sin enfocarse en detalles en específico.
* Versatilidad: Se puede aplicar a una gran variedad de problemas y dominios.
* Enfoque Catálogo: Se basa en un método que ayuda a organizar y clasificar la información de manera clara y lógica dentro de un prompt, asegurando que cada parte tenga un propósito definido y facilite una mejor respuesta de la inteligencia artificial.

### Ventajas
* Eficiencia en tokens: Reduce el número de tokens porque se enfoca en la estructura que definamos en lugar de un contenido detallado.
* Comparación justa: La evaluación se puede enfocar en cómo resolvió el problema, un modelo, en lugar de la forma de la respuesta.
* Eficacia del Zero-shot: Es como un zero-shot donde la influencia del ejemplo en específico es minimizada.

In [41]:
latex_response = model.generate_content("""
Problem Statement:

Problem:
Ana and Carlos went to a candy store. Ana bought 3 chocolates and 2 candies for a total of $8.50. Carlos bought 2 chocolates and 3 candies for a total of $7.50.
Question: How much does each chocolate and each candy cost?

Solution Structure:

Begin the response with "Let’s think step by step."
Follow with the reasoning steps, ensuring the solution process is broken down clearly and logically.
End the solution with the final answer encapsulated in a LaTeX-formatted box, [...], for clarity and emphasis.
Finally, state "The answer is [final answer to the problem].", with the final answer presented in LaTeX notation.
-------------
""").text

print(latex_response)

Let’s think step by step.
Let 'c' be the cost of a chocolate and 'a' be the cost of a candy.
We can set up a system of two equations based on the given information:

Equation 1 (Ana): 3c + 2a = 8.50
Equation 2 (Carlos): 2c + 3a = 7.50

We can solve this system of equations using either substitution or elimination. Let's use elimination.
Multiply Equation 1 by 2: 6c + 4a = 17
Multiply Equation 2 by 3: 6c + 9a = 22.50

Now, subtract the modified Equation 1 from the modified Equation 2:
(6c + 9a) - (6c + 4a) = 22.50 - 17
5a = 5.50
a = 5.50 / 5
a = 1.10

Now that we know the cost of a candy (a = 1.10), we can substitute this value back into either Equation 1 or Equation 2 to find the cost of a chocolate. Let's use Equation 1:
3c + 2(1.10) = 8.50
3c + 2.20 = 8.50
3c = 8.50 - 2.20
3c = 6.30
c = 6.30 / 3
c = 2.10

So, a chocolate costs $2.10 and a candy costs $1.10.

$\boxed{Chocolate: \$2.10, Candy: \$1.10}$

The answer is Chocolate: $2.10, Candy: $1.10.



In [42]:
import re

# Input text (puedes reemplazarlo con cualquier entrada similar)
raw_text = latex_response

def clean_and_convert_to_markdown(text):
    # Remover \n innecesarios y espacios extra
    text = text.replace("\n", " ").strip()

    # Convertir ecuaciones inline $...$ en formato Markdown con MathJax
    text = re.sub(r'\$(.*?)\$', r'\\(\1\\)', text)

    # Separar ecuaciones en bloques con $$ ... $$ para MathJax en Markdown
    text = re.sub(r'\\\((.*?)\\\)', r'$$\1$$', text)

    # Estructurar salida como Markdown
    markdown_template = f"""# Resolución de la Ecuación Cuadrática

{text}
"""

    return markdown_template

# Generar Markdown limpio
markdown_output = clean_and_convert_to_markdown(raw_text)

# Mostrar en Jupyter Notebook
from IPython.display import Markdown
display(Markdown(markdown_output))


# Resolución de la Ecuación Cuadrática

Let’s think step by step. Let 'c' be the cost of a chocolate and 'a' be the cost of a candy. We can set up a system of two equations based on the given information:  Equation 1 (Ana): 3c + 2a = 8.50 Equation 2 (Carlos): 2c + 3a = 7.50  We can solve this system of equations using either substitution or elimination. Let's use elimination. Multiply Equation 1 by 2: 6c + 4a = 17 Multiply Equation 2 by 3: 6c + 9a = 22.50  Now, subtract the modified Equation 1 from the modified Equation 2: (6c + 9a) - (6c + 4a) = 22.50 - 17 5a = 5.50 a = 5.50 / 5 a = 1.10  Now that we know the cost of a candy (a = 1.10), we can substitute this value back into either Equation 1 or Equation 2 to find the cost of a chocolate. Let's use Equation 1: 3c + 2(1.10) = 8.50 3c + 2.20 = 8.50 3c = 8.50 - 2.20 3c = 6.30 c = 6.30 / 3 c = 2.10  So, a chocolate costs $$2.10 and a candy costs $$1.10.  $$\boxed{Chocolate: \$$2.10, Candy: \$$1.10}$$  The answer is Chocolate: $$2.10, Candy: $$1.10.


In [44]:
latex_response = model.generate_content("""
Problem Statement:

Problem: x^2 + 4x + 2 = 0

Solution Structure:

Begin the response with "Let’s think step by step."
Follow with the reasoning steps, ensuring the solution process is broken down clearly and logically.
End the solution with the final answer encapsulated in a LaTeX-formatted box, [...], for clarity and emphasis.
Finally, state "The answer is [final answer to the problem].", with the final answer presented in LaTeX notation.
-------------
""").text

markdown_output = clean_and_convert_to_markdown(latex_response)

# Mostrar en Jupyter Notebook
from IPython.display import Markdown
display(Markdown(markdown_output))

# Resolución de la Ecuación Cuadrática

Let’s think step by step. We are given the quadratic equation $$x^2 + 4x + 2 = 0$$. We can solve this equation using the quadratic formula. The quadratic formula states that for a quadratic equation of the form $$ax^2 + bx + c = 0$$, the solutions for $$x$$ are given by:  $$x = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a}$$  In our case, $$a = 1$$, $$b = 4$$, and $$c = 2$$. Plugging these values into the quadratic formula, we get:  $$x = \frac{-4 \pm \sqrt{4^2 - 4(1)(2)}}{2(1)}$$  $$x = \frac{-4 \pm \sqrt{16 - 8}}{2}$$  $$x = \frac{-4 \pm \sqrt{8}}{2}$$  $$x = \frac{-4 \pm 2\sqrt{2}}{2}$$  $$x = -2 \pm \sqrt{2}$$  Therefore, the two solutions for $$x$$ are $$x = -2 + \sqrt{2}$$ and $$x = -2 - \sqrt{2}$$.  $$\boxed{x = -2 \pm \sqrt{2}}$$  The answer is $$x = -2 \pm \sqrt{2}$$.


### Self-consistency

En lugar de seguir un único camino de razonamiento, la auto-consistencia genera varias soluciones diferentes al mismo problema. Luego, se comparan estas soluciones y se elige la más consistente entre ellas.

In [45]:
response = model.generate_content("""
Un estanque tiene una fuga y pierde agua constantemente. Si se llena por completo,
tarda 6 horas en vaciarse debido a la fuga.
Al mismo tiempo, un grifo está vertiendo agua en el estanque a un ritmo de 10 litros por hora.
Si la capacidad total del estanque es de 60 litros, ¿cuál es la velocidad de la fuga en litros por hora?

Let's think step by step.

1. **Understand the problem statement clearly.**  
   - Identify the key variables and conditions given in the problem.  
   - Determine what needs to be solved or derived.  

2. **Generate multiple independent reasoning paths, at least three.**  
   - Approach the problem from different perspectives.  
   - Consider alternative ways to derive the answer.  
   - Ensure that each method follows a logical and structured reasoning process.  

3. **Compare the solutions obtained from different methods.**  
   - Check if they lead to the same or similar answers.  
   - If there are discrepancies, analyze potential errors or assumptions.  

4. **Select the most consistent answer.**  
   - If multiple methods converge to the same solution, we can be confident in the result.  
   - Clearly state the final answer with justification.  

Finally, the most consistent answer is **___**.

""").text
print(response)

Here's a step-by-step breakdown to solve this problem:

**1. Understand the problem statement clearly.**

*   **Key Variables:**
    *   Time to empty the full pond by leakage = 6 hours
    *   Water filling rate = 10 liters/hour
    *   Pond capacity = 60 liters
*   **Goal:** Find the leakage rate in liters/hour.

**2. Generate multiple independent reasoning paths, at least three.**

*   **Method 1: Focusing on the net change in volume**

    1.  Let 'x' be the leakage rate in liters per hour.
    2.  The pond empties in 6 hours if only leakage is present. So, Leakage rate = Pond capacity / Time to empty = 60 liters / 6 hours = 10 liters/hour.
    3.   Let's rephrase the question: Assume that the leakage rate is 'x' liters per hour and a tap is filling the pool at the rate of 10 litres per hour. The pool gets empty in 6 hours when filled to its max capacity of 60 litres. What is 'x'?
    4. If the tank drains in 6 hours then it drains 60/6 litres per hour which is 10 litres per hour.


In [ ]:
response

# References
* https://www.promptingguide.ai/
* https://platform.openai.com/docs/guides/prompt-engineering
  